In [1]:
import torch
import glob
import os
# glob 결과 숫자 오름차순으로 정리해주는 라이브러리, 기능적으로 필요하지 않았음을 깨달았으나
# 정렬 작업이 유지보수를 가정했을 때 충분히 의미 있다고 생각해서 그냥 놔두기로 함
import natsort
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
from torchvision import models
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import time

In [2]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
import drive.MyDrive.Colab_Notebooks.resnet_datanmodel as datanmodel
import drive.MyDrive.Colab_Notebooks.ResNet as ResNet

Mounted at /content/drive


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [4]:
# 경로 설정, py파일로 변환시 경로는 변경되어야 한다.
# local path
# path=os.path.abspath('../')
# colab path
path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/')

# Resize: 크기를 224, 224로 맞춘다
# ToTensor: 데이터 타입을 Tensor로 만든다. Tensor의 원소는 0~1로 정해진다.(https://pytorch.org/vision/stable/generated/torchvision.transforms.ToTensor.html#torchvision.transforms.ToTensor)
# custom으로 transform를 작성하는 것도 가능하다.
transforms=transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()])

In [5]:
cnd_train=datanmodel.cnd_data(file_path=path, train=True, transforms=transforms)

In [6]:
batch=32
cnd_dataloader=DataLoader(cnd_train, batch_size=batch, shuffle=True)

In [7]:
# 18 model
# train_model=datanmodel.ResNet_compat(blocks_in_model=[2, 2, 2, 2,],
#                                      is_18=True).to(device)
# 34 model
# train_model=datanmodel.ResNet_compat().to(device)
# # 50 model
# train_model=datanmodel.ResNet_compat(blocks_in_model=[3, 4, 6, 3],
#                          layers_in_block=[3, 3, 3, 3],
#                          kernel_sizes=[(1,3,1), (1,3,1), (1,3,1), (1,3,1)],
#                          channel_sizes=[(64,64,256), (128,128,512), (256,256,1024), (512,512, 2048)]).to(device)
###########################################################################
# 34 model(torchvision)- summary()는 선언된 클래스를 불러오기 때문에 instance를 변경해도 변경 사항이 반영되지 않는다.
train_model=models.resnet34().to(device)

# 50 model(torchvision)
# train_model=models.resnet50().to(device)

# torchvision 불러온 모델의 경우 class=2 조정 및 softmax 추가
layer=list(train_model.named_modules())[-1][-1]
layer.out_features=2
softmax=nn.Softmax(-1)
train_model=nn.Sequential(train_model, softmax)
###########################################################################
print(train_model)
summary(train_model, input_size=(3, 224, 224))



Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [8]:
learning_rate=0.01

loss_f= nn.CrossEntropyLoss()
# train_model.parameters: 최적화할 대상의 파라미터
optimizer = torch.optim.SGD(train_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)


In [9]:
log_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_log/')
weight_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_pth/')

In [10]:
writer=SummaryWriter(log_save_path)

In [11]:
# 폴더에 가중치가 있으면 불러온다.
# 아래와 같이 코드가 복잡한 이유는 GPU일 때와 CPU일 때의 차이가 있기 때문에 이를 맞춰줘야 했기 때문
# 또한 병렬 처리가 된 GPU의 경우 추가로 key를 생성하는데, 비병렬 상황에서는 이를 제거해야 된다.
weight_list=natsort.natsorted(glob.glob(weight_save_path+'/*.pth'), reverse=True)

if weight_list:
  start_epoch=int(weight_list[0].split('_')[-1].split('.')[0])+1
  print(f'{start_epoch+1} epoch 부터 시작합니다.')

  # GPU 사용 불가시
  if device=='cpu':
    loaded_weight=torch.load(weight_list[0], map_location=torch.device('cpu'))
    if isinstance(train_model,nn.DataParallel):
      print('cpu 병렬')
    else:
      print('cpu 병렬 x')

  # GPU 사용 가능시
  else:
    loaded_weight=torch.load(weight_list[0])
    if isinstance(train_model,nn.DataParallel):
      print('gpu 병렬')
    else:
      print('gpu 병렬 x')

  model_key=train_model.state_dict().keys()
  weight_key=loaded_weight.keys()

  diff_list=list()
  for key in weight_key:
    if key not in model_key:
      diff_list.append(key)

  for diff_key in diff_list:
    del loaded_weight[diff_key]

  train_model.load_state_dict(loaded_weight)

else:
  start_epoch=0
  print('처음부터 시작합니다.')

처음부터 시작합니다.


In [ ]:
# dogs 11285, 8730, 11675 3588, 5604(not dog), 11853, 2877, 6318, 9078(channel 4), 11410 /3588와 5604가 중복해서 나옴. 특정 데이터 문제일 가능성이 높아짐
# cats 8470, 5686, 9778, 2877, 7276, 11935, 5370
# 위 문제는 비트 수준(bit-depth)문제로 발생한 것이며, 이미 해결함

EPOCHS=40

for epoch in range(start_epoch, EPOCHS):
    # running loss: 5batch동안 loss 누적값
    running_loss=0
    running_acc=0

    # epoch_total_loss: 1 epoch에서 발생한 loss 누적값, 750(1 epoch batch수)을 나눠서 평균 loss값을 구하는데 사용할 예정
    epoch_total_loss=0
    epoch_total_acc=0
    start_time=time.time()


    for i, inp in enumerate(cnd_dataloader):

        input, label= inp
        input, label= input.to(device), torch.Tensor(label).to(device)
        # 모든 gradient를 0으로 설정, 이렇게 하지 않으면 이전 loop의 gradient값이 그대로 남아있어 제대로 학습이 되지 않는다.
        optimizer.zero_grad()

        # train_model을 태운 다음 loss를 계산한다.
        output= train_model(input)
        loss= loss_f(output, label)

        # accuracy를 계산한다.
        correct=0
        for t in zip(output.tolist(), label.tolist()):
          if t[0][0] >= 0.5:
              ans=0
          else:
              ans=1
          if ans==t[1]:
            correct+=1
          else:
            pass

        # loss.backward()로 gradient를 계산하고
        # optimizer를 사용하여 반영한다.
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()
        running_acc+=correct

        if i%5 == 4 and i>0:
            end_time=time.time()

            print(f'Loss [{epoch+1}, {i+1}](epoch, minibatch): ', running_loss/100)
            print(f'Accuracy [{epoch+1}, {i+1}](epoch, minibatch): ', running_acc/(batch*5))
            print('time taken(per 5 batch):', end_time-start_time)
            start_time=end_time
            epoch_total_loss+=running_loss
            epoch_total_acc+=running_acc
            running_loss=0.0
            running_acc=0
    # resnet 34 저장
    torch.save(train_model.state_dict(), os.path.join(weight_save_path, f'model_34_weights_{epoch}.pth'))
    # resnet 50 저장
    # torch.save(train_model.state_dict(), os.path.join(weight_save_path, f'model_50_weights_{epoch}.pth'))
    writer.add_scalar("Loss / epoch ", epoch_total_loss/len(cnd_train), epoch)
    writer.add_scalar("Accuracy / epoch", epoch_total_acc/len(cnd_train), epoch)


writer.close()


Loss [1, 5](epoch, minibatch):  0.34541107177734376
Accuracy [1, 5](epoch, minibatch):  0.4125
time taken(per 5 batch): 77.42042660713196
Loss [1, 10](epoch, minibatch):  0.34540812492370604
Accuracy [1, 10](epoch, minibatch):  0.4625
time taken(per 5 batch): 75.69479942321777
Loss [1, 15](epoch, minibatch):  0.3454069995880127
Accuracy [1, 15](epoch, minibatch):  0.4625
time taken(per 5 batch): 74.1530511379242
Loss [1, 20](epoch, minibatch):  0.345401873588562
Accuracy [1, 20](epoch, minibatch):  0.53125
time taken(per 5 batch): 76.31344938278198
Loss [1, 25](epoch, minibatch):  0.34539730548858644
Accuracy [1, 25](epoch, minibatch):  0.5375
time taken(per 5 batch): 75.86320161819458
Loss [1, 30](epoch, minibatch):  0.34539199352264405
Accuracy [1, 30](epoch, minibatch):  0.55
time taken(per 5 batch): 73.52102375030518
Loss [1, 35](epoch, minibatch):  0.34538590908050537
Accuracy [1, 35](epoch, minibatch):  0.5375
time taken(per 5 batch): 77.70650386810303
Loss [1, 40](epoch, minibat

본문 끝, 테스트 코드

In [ ]:
raise Exception("에러 메시지")

In [ ]:
# 34 model
model_34=datanmodel.ResNet_compat().to(device)
# # 50 model
model_50=datanmodel.ResNet_compat(input_shape=(3, 224, 224),
                        blocks_in_model=[3, 4, 6, 3],
                         layers_in_block=[3, 3, 3, 3],
                         kernel_sizes=[(1,3,1), (1,3,1), (1,3,1), (1,3,1)],
                         channel_sizes=[(64,64,256), (128,128,512), (256,256,1024), (512,512, 2048)]).to(device)

In [ ]:
t_p= sum(p.numel() for p in model_34.parameters())
print(t_p)

In [ ]:
t_p2= sum(p.numel() for p in model_50.parameters())
print(t_p2)

In [ ]:
import torch
import torchvision.models as models

# ResNet-50 모델 인스턴스 생성
model = models.resnet50()

# 모델의 파라미터 개수 계산
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

In [ ]:
# ResNet-50 모델 인스턴스 생성
model = models.resnet34()

# 모델의 파라미터 개수 계산
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

In [ ]:
# ResNet-50 모델 인스턴스 생성
model = models.resnet18()

# 모델의 파라미터 개수 계산
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

In [ ]:
train_model=datanmodel.ResNet_compat(blocks_in_model=[3, 4, 6, 3],
                         layers_in_block=[3, 3, 3, 3],
                         kernel_sizes=[(1,3,1), (1,3,1), (1,3,1), (1,3,1)],
                         channel_sizes=[(64,64,256), (128,128,512), (256,256,1024), (512,512, 2048)]).to(device)

t_p3= sum(p.numel() for p in train_model.parameters())
print(t_p3)

In [ ]:
model_50=models.resnet50()
summary(model_50, input_size=(3, 224, 224))

In [ ]:
import torch
import torchvision.models as models
from torchsummary import summary

# ResNet-50 모델 불러오기
model = models.resnet50().to(device)

# 모델 요약 정보 출력
summary(model, (3, 224, 224))  # 입력 이미지 크기에 맞게 설정

In [ ]:
model_34 = models.resnet34().to(device)
for i in model_34.named_modules():
  print(i)

In [ ]:
layer=list(model_34.named_modules())[-1][-1]
layer.out_features=2

print(model_34)

